## Movie Recommendation Project

### The Beginning

In [249]:
# import the libraries
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import matplotlib.pyplot as plt

In [152]:
#read in the data
df = pd.read_csv(r'C:\Users\joshu\CSV Files\movies.csv')

In [203]:
df.head(500)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,Unnamed: 1255,Unnamed: 1256,Unnamed: 1257,Unnamed: 1258,Unnamed: 1259,Unnamed: 1260,Unnamed: 1261,Unnamed: 1262,Unnamed: 1263,Unnamed: 1264
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.31295,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,490,80000000,Thriller Science Fiction Adventure Action,http://asoundofthunder.warnerbros.com/,10077,dying and death time travel romance dinosaur,en,A Sound of Thunder,When a hunter sent back to the prehistoric era...,7.342892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,491,130000000,Action Adventure History Romance Drama,NaN,76649,gladiator arena gladiator fight lava roman,en,Pompeii,"Set in 79 A.D., POMPEII tells the epic story o...",50.561849,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,492,8000000,Comedy Animation,NaN,293644,3d,es,Don Gato: El inicio de la pandilla,Top Cat has arrived to charm his way into your...,0.719996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,493,60000000,Drama Romance,http://www.abeautifulmind.com/,453,individual schizophrenia massachusetts love of...,en,A Beautiful Mind,"At Princeton University, John Nash struggles t...",59.248437,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [154]:
df.shape

(4809, 1265)

In [155]:
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']

### Data Cleaning

In [156]:
# handling nulls
for i in selected_features:
    df[i] = df[i].fillna('')
df['title'] = df['title'].fillna('')

### Data Exploration

In [278]:
# looks at a list of all words in df['ketywords']
keywords = df['genres'].tolist()
result = []
for i in keywords:
    splitted = i.split()
    for j in splitted:
        result.append(j)

In [279]:
# creates a dictionary of frequencies for these words
dictionary = {}
for i in result:
    if i in dictionary:
        dictionary[i] += 1
    else:
        dictionary[i] = 1
print (dictionary)

{'Action': 1153, 'Adventure': 790, 'Fantasy': 418, 'Science': 530, 'Fiction': 530, 'Crime': 696, 'Drama': 2297, 'Thriller': 1259, 'Animation': 234, 'Family': 510, 'Western': 80, 'Comedy': 1722, 'Romance': 890, "'credit_id':": 3, "'55492449c3a36841b2000363'": 1, 'Horror': 519, 'Mystery': 347, 'History': 197, 'War': 142, "'id':": 1, '1767009}': 1, "{'name':": 2, "'Michael": 1, "Parks'": 1, 'Music': 183, "'5681e6bfc3a36860e9011ee5'": 1, "'58f5f6d1925141277100b78b'": 1, 'Documentary': 110, "'Daniel": 1, "Gordon'": 1, 'Foreign': 34, 'TV': 8, 'Movie': 8}


### Creating Text

In [157]:
features = df['genres'] + ' ' + df['keywords'] + ' ' + df['tagline'] + ' ' + df['cast'] + ' ' + df['director']

In [158]:
print(features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4804    Action Crime Thriller united states\u2013mexic...
4805    Comedy Romance  A newlywed couple's honeymoon ...
4806    Comedy Drama Romance TV Movie date love at fir...
4807      A New Yorker in Shanghai Daniel Henney Eliza...
4808    Documentary obsession camcorder crush dream gi...
Length: 4809, dtype: object


### Convert to Numerical Values

In [159]:
vectorizer = TfidfVectorizer()

In [160]:
feature_vectors = vectorizer.fit_transform(features)

In [161]:
print(feature_vectors)

  (0, 2438)	0.1727269234969866
  (0, 7763)	0.11281946761216571
  (0, 13033)	0.19423432006987434
  (0, 10237)	0.16059231591328046
  (0, 8764)	0.22708109846346383
  (0, 14617)	0.15151416868595385
  (0, 16677)	0.19842983735418923
  (0, 14073)	0.205956457941733
  (0, 13328)	0.2177400340451317
  (0, 17299)	0.2019755488126334
  (0, 17016)	0.23642216288179602
  (0, 13358)	0.15022036822291404
  (0, 11511)	0.27209420901160897
  (0, 11200)	0.09051396620449952
  (0, 17007)	0.1282251636064218
  (0, 15270)	0.0709833692900418
  (0, 4952)	0.24024658931920426
  (0, 14280)	0.21391560760772346
  (0, 3231)	0.24958765373753639
  (0, 16596)	0.12550744845505796
  (0, 14387)	0.3396344162602317
  (0, 5843)	0.16467965954169428
  (0, 3071)	0.22207581114012026
  (0, 3685)	0.21391560760772346
  (0, 5444)	0.10365929555963475
  :	:
  (4807, 17275)	0.28866712901660546
  (4807, 4842)	0.2471929040013495
  (4807, 410)	0.17732764087392294
  (4807, 6942)	0.28866712901660546
  (4807, 11671)	0.21562869602539067
  (4807, 16

### Find Similarity Scores

In [162]:
similarity = cosine_similarity(feature_vectors)

In [163]:
print(similarity)
print(similarity.shape)

[[1.         0.0722257  0.03774545 ... 0.         0.         0.        ]
 [0.0722257  1.         0.03282986 ... 0.03576038 0.         0.        ]
 [0.03774545 0.03282986 1.         ... 0.         0.05370221 0.        ]
 ...
 [0.         0.03576038 0.         ... 1.         0.         0.02651822]
 [0.         0.         0.05370221 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651822 0.         1.        ]]
(4809, 4809)


### Handling User Input

In [207]:
movie_name = input("Enter Your Favorite Movie: ")

Enter Your Favorite Movie: Iron man


In [280]:
#List of movies in dataset
titles = df['title'].tolist()

In [209]:
# find the matches
find_close_match = difflib.get_close_matches(movie_name, titles)
print (find_close_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [210]:
# find the legitimate closest match
close_match = find_close_match[0]
print (close_match)

Iron Man


In [213]:
#get the index of the movie
the_index = df[df.title == close_match].index[0]
print (the_index)

69


### The Algorithm

In [215]:
similarity_score = list(enumerate(similarity[int(the_index)]))
print (similarity_score)

[(0, 0.03358379103481251), (1, 0.05465009729745318), (2, 0.013743167359589824), (3, 0.006472407458075143), (4, 0.03270206570382855), (5, 0.013914206795630699), (6, 0.07693825481438513), (7, 0.23945283538298054), (8, 0.007886442277287633), (9, 0.0759979593639291), (10, 0.07537428591211721), (11, 0.011932267618466426), (12, 0.013714561024445758), (13, 0.012382423322785168), (14, 0.0965825145754013), (15, 0.007289559360676009), (16, 0.2270554756713479), (17, 0.01311962727035421), (18, 0.04141648041130168), (19, 0.07883371701219326), (20, 0.07981724556148166), (21, 0.011272671062879127), (22, 0.006895558259187204), (23, 0.006602345328342218), (24, 0.012670332970512954), (25, 0.0), (26, 0.21567278301132473), (27, 0.030593877143290968), (28, 0.06453933511362897), (29, 0.0), (30, 0.014054021485629911), (31, 0.0807771825315426), (32, 0.31466830471914936), (33, 0.028786440767309778), (34, 0.13090912583212602), (35, 0.0), (36, 0.03536476251917614), (37, 0.03186639292236786), (38, 0.0080278694443

In [216]:
len(similarity_score)

4809

In [221]:
# sort movies based on similarity score
sorted_sims = sorted(similarity_score, key = lambda x:x[1], reverse=True)

### Many Examples of the Full Code

In [277]:
# asks for your favorite movie
movie_name = input("Enter Your Favorite Movie: ")

titles = df['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, titles)

# returns the closest match to your input (accounting for spelling errors)
close_match = find_close_match[0]

# finds the index of this match and produces a list of simialrity scores to the other indeces
the_index = df[df.title == close_match].index[0]

similarity_score = list(enumerate(similarity[int(the_index)]))

# sorts the list of similarity scores
sorted_sims = sorted(similarity_score, key = lambda x:x[1], reverse=True)

print ("These are the movies suggested for you: ")

#prints the movies with the 10 highest similarity scores
for i in range(10):
    index = sorted_sims[i][0]
    title_from_index = df[df.index==index]['title'].values[0]
    print (str(i+1) + '.', title_from_index)

Enter Your Favorite Movie: Iron Man
These are the movies suggested for you: 
1. Iron Man
2. Iron Man 2
3. Iron Man 3
4. Avengers: Age of Ultron
5. The Avengers
6. Captain America: Civil War
7. Captain America: The Winter Soldier
8. Ant-Man
9. X-Men
10. Made


In [273]:
movie_name = input("Enter Your Favorite Movie: ")

titles = df['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, titles)

close_match = find_close_match[0]

the_index = df[df.title == close_match].index[0]

similarity_score = list(enumerate(similarity[int(the_index)]))

sorted_sims = sorted(similarity_score, key = lambda x:x[1], reverse=True)

print ("These are the movies suggested for you: ")

for i in range(10):
    index = sorted_sims[i][0]
    title_from_index = df[df.index==index]['title'].values[0]
    print (str(i+1) + '.', title_from_index)

Enter Your Favorite Movie: The Godfather
These are the movies suggested for you: 
1. The Godfather
2. The Godfather: Part III
3. Apocalypse Now
4. Closer
5. The Godfather: Part II
6. Mickey Blue Eyes
7. August Rush
8. Leaving Las Vegas
9. Machete
10. Dracula


In [274]:
movie_name = input("Enter Your Favorite Movie: ")

titles = df['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, titles)

close_match = find_close_match[0]

the_index = df[df.title == close_match].index[0]

similarity_score = list(enumerate(similarity[int(the_index)]))

sorted_sims = sorted(similarity_score, key = lambda x:x[1], reverse=True)

print ("These are the movies suggested for you: ")

for i in range(10):
    index = sorted_sims[i][0]
    title_from_index = df[df.index==index]['title'].values[0]
    print (str(i+1) + '.', title_from_index)

Enter Your Favorite Movie: Caddyshack
These are the movies suggested for you: 
1. Caddyshack
2. Happy Gilmore
3. Tin Cup
4. National Lampoon's Vacation
5. Groundhog Day
6. Stripes
7. Jarhead
8. Memoirs of an Invisible Man
9. Year One
10. Modern Problems


In [275]:
movie_name = input("Enter Your Favorite Movie: ")

titles = df['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, titles)

close_match = find_close_match[0]

the_index = df[df.title == close_match].index[0]

similarity_score = list(enumerate(similarity[int(the_index)]))

sorted_sims = sorted(similarity_score, key = lambda x:x[1], reverse=True)

print ("These are the movies suggested for you: ")

for i in range(10):
    index = sorted_sims[i][0]
    title_from_index = df[df.index==index]['title'].values[0]
    print (str(i+1) + '.', title_from_index)

Enter Your Favorite Movie: Star Wars
These are the movies suggested for you: 
1. Star Wars
2. The Empire Strikes Back
3. Return of the Jedi
4. On Her Majesty's Secret Service
5. Titan A.E.
6. Star Wars: Episode I - The Phantom Menace
7. The Helix... Loaded
8. Star Wars: Episode III - Revenge of the Sith
9. Timecop
10. Lawrence of Arabia


In [276]:
movie_name = input("Enter Your Favorite Movie: ")

titles = df['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, titles)

close_match = find_close_match[0]

the_index = df[df.title == close_match].index[0]

similarity_score = list(enumerate(similarity[int(the_index)]))

sorted_sims = sorted(similarity_score, key = lambda x:x[1], reverse=True)

print ("These are the movies suggested for you: ")

for i in range(10):
    index = sorted_sims[i][0]
    title_from_index = df[df.index==index]['title'].values[0]
    print (str(i+1) + '.', title_from_index)

Enter Your Favorite Movie: Toy Story
These are the movies suggested for you: 
1. Toy Story
2. Toy Story 2
3. Toy Story 3
4. Cars 2
5. Cloud Atlas
6. Home
7. Larry Crowne
8. Quest for Camelot
9. Big
10. That Thing You Do!
